In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import io
from google.colab import drive
import threading
import time
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import glob
glob.glob("/content/drive/My Drive/google_notebook/M5_traindata2/*")

['/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sample_submission.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv',
 '/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a']

In [0]:
import os.path

train_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sales_train_validation.csv")
calendar_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar.csv")
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
calendar_data2 = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/calendar2.csv")

days_num = 1913

cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap"] #"price", "price_down", "price_up", "sold", "value"は後程追加
train_data2a0 = pd.DataFrame(index=[ij for ij in range(days_num)], columns=cols)

start_n = 0

thread_num = 5 #スレッド数

print(len(train_data))

current_num = [-1 for jj in range(thread_num)]  # 実行中のファイル番号(重複処理を防ぐ)


class traindata2Threading(threading.Thread):

    def __init__(self, thread_name):
        self.thread_name = str(thread_name)
        self.thread_number = int(thread_name)
        threading.Thread.__init__(self)

    def __str__(self):
        return self.thread_name

    def stap(self, ST):
        return sellprices2_data["snap_" + ST]

    def run(self):
        for ii in range(start_n, len(train_data)):
            if ((ii in current_num) == False): #その行は処理されていない
                current_num[self.thread_number] = ii
                if (os.path.isfile("/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv") == False):  # ファイルがまだ作成されていない
                    train_data2a = train_data2a0.copy()
                    tra_sir = train_data.iloc[ii]
                    for jj in range(days_num):train_data2a.iloc[jj,0:6] = tra_sir[0:6]
                    train_data2a.iloc[:,6] = calendar_data.iloc[:,1] #週
                    train_data2a.iloc[:,7:11] = calendar_data.iloc[:,3:7] #年, 月, 曜日
                    train_data2a.iloc[:,11:15] = calendar_data2.iloc[:,1:5] #イベント情報
                    train_data2a.iloc[:,15] = self.stap(tra_sir[5]) #snap

                    sap_mindata = sellprices2_data[sellprices2_data.item_id == tra_sir[1]]  # sellpricesの中から必要なデータを抽出
                    sap_mindata = sap_mindata[sap_mindata.store_id == tra_sir[4]]
                    sap_mindata = sap_mindata.drop(["store_id", "item_id"], axis=1)

                    train_data2a = pd.merge(train_data2a, sap_mindata, on='wm_yr_wk', how='left') #価格とその変化

                    is_sold = lambda x: 0 if x is None else 1 #価格データがあれば1, そうでなければ0
                    train_data2a["sold"] = train_data2a["sell_price"].map(is_sold)


                    train_data2a.to_csv(
                        "/content/drive/My Drive/google_notebook/M5_traindata2/train_data2a/train2a_" + str(ii) + ".csv", index=False)
                current_num[self.thread_number] = -1
                print('\r', end='')
                print(current_num, end='')

thread_list = []
for i in range(thread_num):
    thread = traindata2Threading(thread_name=i)
    thread.start()
    thread_list.append(thread)
    sleep_seconds = 1
    time.sleep(sleep_seconds)

for thread in thread_list:
    thread.join()

In [4]:
cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk", "wday", "month", "year", "d",
        "Religious", "National", "Cultural", "Sporting", "snap", "price", "price_down", "price_up", "sold", "value"]
train_data2a0 = pd.DataFrame(index=[ij for ij in range(1913)], columns=cols)
train_data2a0

,id,item_id,dept_id,cat_id,store_id,state_id,wm_yr_wk,wday,month,year,d,Religious,National,Cultural,Sporting,snap,price,price_down,price_up,value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
sellprices2_data = pd.read_csv("/content/drive/My Drive/google_notebook/M5_traindata2/sell_prices2.csv")
sellprices2_data

,store_id,item_id,wm_yr_wk,sell_price,price_down,price_up
0,CA_1,HOBBIES_1_001,11325,9.58,0,0
1,CA_1,HOBBIES_1_001,11326,9.58,1,0
2,CA_1,HOBBIES_1_001,11327,8.26,2,0
3,CA_1,HOBBIES_1_001,11328,8.26,0,0
4,CA_1,HOBBIES_1_001,11329,8.26,0,0
...,...,...,...,...,...,...
1048570,CA_2,HOUSEHOLD_2_446,11527,25.97,0,0
1048571,CA_2,HOUSEHOLD_2_446,11528,25.97,0,0
1048572,CA_2,HOUSEHOLD_2_446,11529,25.97,0,0
1048573,CA_2,HOUSEHOLD_2_446,11530,25.97,0,0
